# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In this tutorial we use the defaul version of the CTGAN:
https://github.com/sdv-dev/CTGAN

We also need to install table evaluator to evaluate the quality of the generated synthetic data.

In [ ]:
%pip install ctgan


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
%pip install table_evaluator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 26.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.3 which is incompatible.


Now we download a dataset that includes both continuous and discrete values. I chose a random dataset from Kaggle: https://www.kaggle.com/datasets/mirichoi0218/insurance

In [ ]:
!gdown 1cwOlGDM1JG3Nn-SWXBL2Slh5hsiEE_th

Downloading...
From: https://drive.google.com/uc?id=1cwOlGDM1JG3Nn-SWXBL2Slh5hsiEE_th
To: /content/UNSW_NB15_training-set.csv
100% 15.4M/15.4M [00:00<00:00, 133MB/s]


In [ ]:
import pandas as pd
data = pd.read_csv('./ToN_IoT_1.csv')
selected_columns = ['src_port', 'dst_port', 'proto', 'service', 'duration', 'src_bytes', 'dst_bytes',
                    'conn_state', 'missed_bytes', 'src_pkts', 'src_ip_bytes', 'dst_pkts', 'dst_ip_bytes',
                    'weird_name', 'weird_addl', 'weird_notice','label', 'type']
selected_samples = pd.concat([
    data[data['type'] == 'normal'].sample(n=500, random_state=42)[selected_columns],
    data[data['type'] == 'backdoor'].sample(n=500, random_state=42)[selected_columns],
    data[data['type'] == 'ddos'].sample(n=500, random_state=42)[selected_columns],
    data[data['type'] == 'dos'].sample(n=500, random_state=42)[selected_columns],
    data[data['type'] == 'injection'].sample(n=500, random_state=42)[selected_columns]
])


In [ ]:
data['type'].value_counts()

normal       34955
backdoor     14102
dos          14094
ddos         13981
injection    13975
Name: type, dtype: int64

In [ ]:
selected_samples

,src_port,dst_port,proto,service,duration,src_bytes,dst_bytes,conn_state,missed_bytes,src_pkts,src_ip_bytes,dst_pkts,dst_ip_bytes,weird_name,weird_addl,weird_notice,label,type
16343,43589,53,udp,dns,0.000873,41,89,SF,0,1,69,1,117,-,-,-,0,normal
6569,1750,1750,tcp,-,0.240355,0,0,S0,0,120,4800,0,0,-,-,-,0,normal
82525,56995,53,udp,dns,0.000497,41,89,SF,0,1,69,1,117,-,-,-,0,normal
85771,5353,5353,udp,dns,4.251187,1946,0,S0,0,10,2426,0,0,-,-,-,0,normal
27212,35967,53,udp,-,0.000000,0,0,OTH,0,0,0,0,0,-,-,-,0,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50937,47492,80,tcp,http,7.166592,252,2167,SF,0,6,572,6,2487,-,-,-,1,injection
33186,53322,80,tcp,http,1.365644,266,12183,SF,0,6,586,12,12815,-,-,-,1,injection
6743,37132,80,tcp,http,2.726459,213,3440,SF,0,5,481,6,3760,-,-,-,1,injection
30598,45864,80,tcp,http,0.835104,248,524,SF,0,5,468,4,696,-,-,-,1,injection


Next, we define a list with column names for categorical variables. This list will be passed to the model so that the model can decide how to process these fields.

In [ ]:
categorical_features = ['src_port', 'dst_port', 'proto', 'service', 'duration', 'src_bytes', 'dst_bytes', 'conn_state', 'missed_bytes', 'src_pkts', 'src_ip_bytes', 'dst_pkts', 'dst_ip_bytes','weird_name', 'weird_addl', 'weird_notice','label', 'type']


#Model training

Next, we simply define an instance of CTGANSynthesizer and call the fit method with the dataframe and the list of categorical variables.

We train the model for 300 epochs only as the discriminator and generator loss becomes quite low after these many epochs.

In [ ]:
from ctgan import CTGAN

ctgan = CTGAN(verbose=True)
ctgan.fit(selected_samples, categorical_features,epochs=100)

Gen. (0.78) | Discrim. (0.01): 100%|██████████| 100/100 [59:00<00:00, 35.41s/it]


#Synthetic data generation

In [ ]:
samples = pd.DataFrame()

# Số lượng mẫu cho mỗi loại tấn công
num_samples_per_attack = 1000

# Lặp qua từng loại tấn công và lấy mẫu ngẫu nhiên
for attack_type in ['injection', 'dos', 'normal', 'ddos', 'backdoor']:
    # Lấy mẫu ngẫu nhiên từ DataFrame gốc và thêm vào samples
    samples_for_attack = data[data['type'] == attack_type].sample(n=num_samples_per_attack, random_state=42)
    samples = pd.concat([samples, samples_for_attack])

In [ ]:
samples['type'].value_counts()

type
injection    1000
dos          1000
normal       1000
ddos         1000
backdoor     1000
Name: count, dtype: int64

In [ ]:
import os

# Đường dẫn tới tệp CSV đích
output_file = 'ToN_IoT_2.csv'

# Kiểm tra xem tệp tồn tại không
if os.path.isfile(output_file):
    # Nếu tệp đã tồn tại, đọc dữ liệu từ tệp đích vào DataFrame
    existing_data = pd.read_csv(output_file)

    # Sắp xếp lại cột của DataFrame samples sao cho chúng giống với DataFrame existing_data
    samples = samples.reindex(columns=existing_data.columns)

    # Ghi dữ liệu mới vào tiếp theo mà không ghi đè
    samples.to_csv(output_file, mode='a', header=False, index=False)
else:
    # Nếu tệp không tồn tại, ghi dữ liệu mới vào tệp mới
    samples.to_csv(output_file, index=False)


In [ ]:
data_new = pd.read_csv('./ToN_IoT_2.csv')
data_new

,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,conn_state,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,192.168.1.33,4444,192.168.1.193,49207,tcp,-,0.000000,0,0,OTH,...,0,0,-,-,-,-,-,-,1,ransomware
1,192.168.1.33,7870,192.168.1.193,49158,tcp,-,0.000000,0,0,OTH,...,0,0,-,-,-,-,-,-,1,ransomware
2,192.168.1.31,60186,192.168.1.195,80,tcp,http,0.013428,155,978,SF,...,0,0,-,-,-,-,-,-,1,password
3,192.168.1.33,35977,192.168.1.193,445,tcp,-,0.000000,0,0,OTH,...,0,0,-,-,-,-,-,-,1,ransomware
4,192.168.1.32,39866,18.194.169.124,80,tcp,-,4.568160,0,0,SH,...,0,0,-,-,-,-,-,-,1,xss
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61618,192.168.1.193,49252,192.168.1.33,8080,tcp,-,0.000125,0,0,REJ,...,0,0,-,-,-,-,-,-,1,backdoor
61619,192.168.1.193,50117,192.168.1.33,80,tcp,-,0.000207,0,0,REJ,...,0,0,-,-,-,-,-,-,1,backdoor
61620,192.168.1.193,49469,192.168.1.33,80,tcp,-,0.000118,0,0,REJ,...,0,0,-,-,-,-,-,-,1,backdoor
61621,192.168.1.193,49417,192.168.1.37,8080,tcp,-,0.000019,0,0,REJ,...,0,0,-,-,-,-,-,-,1,backdoor


In [ ]:
data_new['type'].value_counts()

type
password      14028
ransomware    13993
xss           13974
scanning      13907
injection      1000
dos            1000
normal         1000
ddos           1000
backdoor       1000
mitm            721
Name: count, dtype: int64

#Evaluation

In [ ]:
selected_data = data[selected_columns]
selected_data

,src_port,dst_port,proto,service,duration,src_bytes,dst_bytes,conn_state,missed_bytes,src_pkts,src_ip_bytes,dst_pkts,dst_ip_bytes,weird_name,weird_addl,weird_notice,label,type
0,45676,53,udp,dns,0.003166,54,54,SF,0,1,82,1,82,-,-,-,1,ddos
1,37914,80,tcp,http,0.339130,337,1919,SF,0,7,709,5,2187,-,-,-,1,injection
2,1189,1189,tcp,-,0.247497,0,0,S0,0,120,4800,0,0,-,-,-,0,normal
3,45006,443,tcp,-,0.000322,0,0,S1,0,2,112,1,60,-,-,-,1,ddos
4,60071,53,udp,dns,0.000009,74,90,SF,0,2,130,2,146,-,-,-,1,ddos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91102,74,74,tcp,-,0.008967,0,0,REJ,0,4,160,4,160,-,-,-,1,dos
91103,137,137,udp,dns,1.503722,300,0,S0,0,6,468,0,0,-,-,-,0,normal
91104,39134,53,udp,dns,0.012897,41,89,SF,0,1,69,1,117,-,-,-,0,normal
91105,53648,53,udp,dns,0.000026,130,0,S0,0,2,186,0,0,-,-,-,0,normal


In [ ]:
samples

,src_port,dst_port,proto,service,duration,src_bytes,dst_bytes,conn_state,missed_bytes,src_pkts,src_ip_bytes,dst_pkts,dst_ip_bytes,weird_name,weird_addl,weird_notice,label,type
0,19911,80,tcp,http,0.000413,236,0,RSTO,0,2,104,1,696,NaN,NaN,NaN,1,dos
1,35116,51782,udp,dns,0.000140,0,112,OTH,94900,16,620,5,1664,NaN,NaN,NaN,0,normal
2,37010,40,udp,<NA>,60.876147,233,1124,SF,0,91,785,12,112,NaN,NaN,NaN,1,normal
3,49703,80,tcp,<NA>,160.420785,0,1486,S0,0,6,48,1,40,NaN,NaN,NaN,1,backdoor
4,42856,80,tcp,<NA>,0.000019,39,0,SF,0,120,48,0,3812,NaN,NaN,NaN,1,dos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,83,80,tcp,<NA>,1.264965,43,0,SF,0,0,164,6,146,NaN,NaN,NaN,1,injection
996,51419,80,tcp,<NA>,0.000111,226,0,SF,0,2,40,1,40,NaN,NaN,NaN,1,backdoor
997,35902,79,tcp,http,0.000110,284,3440,REJ,0,7,4560,2,86,NaN,NaN,NaN,1,normal
998,37,80,tcp,http,0.000030,0,212,RSTO,0,5,474,3,3760,NaN,NaN,NaN,1,injection


In [ ]:
from table_evaluator import TableEvaluator

# Tạo TableEvaluator với các cột đã chọn
table_evaluator = TableEvaluator(selected_data, samples, cat_cols=categorical_features)

# Thực hiện đánh giá và hiển thị kết quả
table_evaluator.visual_evaluation()


In [2]:
import pandas as pd
data1=pd.read_csv("./ToN_IoT_1.csv")
data2=pd.read_csv("./ToN_IoT_2.csv")

In [3]:
data1['type'].value_counts()

normal        17419
xss            7058
ransomware     7056
password       7054
ddos           7047
injection      7005
backdoor       6965
scanning       6958
dos            6927
mitm            376
Name: type, dtype: int64

In [4]:
data2['type'].value_counts()

normal        17536
dos            7167
backdoor       7137
password       6974
injection      6970
scanning       6949
ransomware     6937
ddos           6934
xss            6916
mitm            345
Name: type, dtype: int64